# TV Script Generation Using Recurrent Neural Networks
---
<img src="assets/books.jpg">

In this project, we will generate our own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using recurrent neural networks (RNNs).  We will use a part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The RNN model will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## 1.0. Load in the Data

First, we define `load_data` function to load in the data. 

In [1]:
import os

def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

In the code cell below, we will load the data from `./data/Seinfeld_Scripts.txt`.

In [2]:
# load in data
data_dir = './data/Seinfeld_Scripts.txt'
text = load_data(data_dir)

### Explore the Data

In [3]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## 2.0. Pre-processing Data
The first thing to do to any dataset is pre-processing.  We will implement the following pre-processing functions:
- Lookup Table
- Tokenize Punctuation

### 2.1. Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, we create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

We return these dictionaries in the **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    # Count the number of each word in the text
    word_counts = Counter(text)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create vocab_to_int dictionary
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)

### 2.2. Tokenize Punctuation
We will be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Thus, we will implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". We will create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. 

In [5]:
from string import punctuation

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token = {'.': '||Period||', 
             ',': '||Comma||', 
             '"': '||Quotation_Mark||', 
             ';': '||Semicolon||',
             '!': '||Exclamation_Mark||',
             '?': '||Question_Mark||', 
             '(': '||Left_Parentheses||', 
             ')': '||Right_Parentheses||',
             '-': '||Dash||', 
             '\n':'||Return||'}    
    
    return token

### 2.3. Putting It All Together
We define `preprocess_and_save_data` function, which combine `create_lookup_tables` and `token_lookup` function, to pre-process and save the data.

In [6]:
import pickle

def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    text = load_data(dataset_path)
    
    # Ignore notice, since we don't use it for analysing the data
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()
    
    SPECIAL_WORDS = {'PADDING': '<PAD>'}

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

#### Pre-process all the data and save it

We run the code cell below to pre-process all the data and save it to file. 

In [7]:
# pre-process training data
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

#### Checkpoint
We can load the preprocessed data that have been saved to disk.

In [8]:
import os
import pickle
import torch

def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

In [9]:
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

## 3.0. Build the Neural Network
In this section, we will build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

#### Check Access to GPU

In [10]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

### 3.1. Define Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

In [11]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """

    features = []
    targets = []
    
    for idx in range(0, len(words)):
        if (idx + sequence_length) < len(words):
            feature, tensor = [], []
            feature = words[idx:idx+sequence_length]
            tensor = words[idx+sequence_length]
            features.append(feature)
            targets.append(tensor)
    
    features = np.asarray(features)
    targets = np.asarray(targets)
    
    # Create a Tensor dataset
    feature_tensors = TensorDataset(torch.from_numpy(features), torch.from_numpy(targets))
    dataloader = DataLoader(feature_tensors, shuffle=True, batch_size=batch_size)
    
    # return a dataloader
    return dataloader

### 3.2. Define RNN Architecture
To develop an RNN model, we will have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

In this notebook, we use embedding and LSTM layers to build the RNN model. The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we will need a more efficient representation for our input data than one-hot encoded vectors. 

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in our data. 

>**Finally, the LSTM outputs will go to a output layer.** We are using a fully-connected neural network layer.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

In [12]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        
        # linear
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # fully-connected layer
        out = self.fc(lstm_out)
        
        # reshape to be batch_size first
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1] # get last batch of labels
        
        # return one batch of output word scores and the hidden state
        return out, hidden  
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

### 3.3. Define Forward and Backpropagation

We use the RNN class we implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop. 

In [13]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden, clip=5):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])
    
    # zero accumulated gradients
    rnn.zero_grad()
    
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
    
    # get the output from the model
    output, h = rnn(inp, h)
    
    # calculate the loss and perform backprop
    loss = criterion(output, target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

### 3.4. Train Neural Network

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

The training loop is implemented for in the `train_rnn` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. We will set this parameter along with other parameters in the next section.

In [14]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure we iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

#### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of the RNN.
- Set `n_layers` to the number of layers/cells in the RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

In [15]:
# Data params
# Sequence Length
sequence_length = 10 # of words in a sequence
# Batch Size
batch_size = 128

# data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [16]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 250
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

#### Train

In [17]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.517633271217346

Epoch:    1/10    Loss: 4.897500102043152

Epoch:    1/10    Loss: 4.658834692001343

Epoch:    1/10    Loss: 4.53312217092514

Epoch:    1/10    Loss: 4.438450860977173

Epoch:    1/10    Loss: 4.393903307914734

Epoch:    1/10    Loss: 4.351630313873291

Epoch:    1/10    Loss: 4.28318585729599

Epoch:    1/10    Loss: 4.282118978977203

Epoch:    1/10    Loss: 4.23600855922699

Epoch:    1/10    Loss: 4.213591070175171

Epoch:    1/10    Loss: 4.192781659126282

Epoch:    1/10    Loss: 4.1700753741264345

Epoch:    2/10    Loss: 4.082435155923645

Epoch:    2/10    Loss: 3.9608580741882324

Epoch:    2/10    Loss: 3.9974955320358276

Epoch:    2/10    Loss: 3.9675925345420837

Epoch:    2/10    Loss: 3.9525283479690554

Epoch:    2/10    Loss: 3.94262863111496

Epoch:    2/10    Loss: 3.9326539006233214

Epoch:    2/10    Loss: 3.9325126857757566

Epoch:    2/10    Loss: 3.929595118522644

Epoch:    2/10    Loss:

#### Save the model
After running the above training cell, the trained model will be saved by name, `trained_rnn`.

In [20]:
def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)

In [21]:
# saving the trained model
save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


#### Checkpoint
We can load the trained model that have been saved to disk.

In [22]:
import os
import pickle
import torch

def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

In [23]:
_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
trained_rnn = load_model('./save/trained_rnn')

## 4.0. Generate TV Script
With the network trained and saved, we will use it to generate a new, "fake" Seinfeld TV script in this section.

### 4.1. Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. We will be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [24]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### 4.2. Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script we want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

We can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script.

In [25]:
# run the cell multiple times to get different results
gen_length = 400 # modify the length
prime_word = 'jerry' # name for starting the script

SPECIAL_WORDS = {'PADDING': '<PAD>'}
pad_word = SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: i was a little curious to see him in the apartment.

george: oh, yeah, that's right.

jerry:(to george) you know, i'm not going back in this place.

jerry: i was not in the neighborhood and the other night, i think i was a little eccentric and i have a little fun about that. i'm gonna be a big fan for this, but i don't want to see him anymore.

jerry:(to george) i was wondering you were going to have the same thing.

george: i don't know.

elaine: what? what do you think, you can see her about the car?

elaine: no, i can't.

elaine: oh, no, i'm sorry. i got the one...

jerry:(to jerry) what did she say about the subject for?

jerry: i don't want to hear this.

elaine: what?

kramer: yeah, yeah, yeah, but it's the damnedest thing i ever said to you.

george: you know, i'm going to be the only person who sold it.

elaine: oh, no, i don't think so.

kramer:(to the phone) oh, come on, you got to do me a favour, and i'm just going to do something like this.

jerry: i can't believe yo

#### Save our favorite scripts

Once we have a script that we like (or find interesting), save it to a text file!

In [26]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()